In [1]:
import chess
import numpy as np
import pandas as pd
import pickle

In [2]:


# Step one: get a csv of 100k text games
input_file = '100k_games.csv'

df = pd.read_csv(input_file, usecols=['transcript'])
df.to_csv('100k_transcripts.csv', index=False)

input_file = '100k_transcripts.csv'
df = pd.read_csv(input_file)

def format_transcript(game: str) -> str:
    new_game = ';' + game.split('\n\n')[1]
    return new_game

df['transcript'] = df['transcript'].apply(format_transcript)
df.to_csv('100k_transcripts.csv', index=False)

input_file = '100k_transcripts.csv'
df = pd.read_csv(input_file)

for game in df.head()['transcript']:
    print(game)
    print()

;1.e4 c5 2.Nf3 Nc6 3.Bb5 Nf6 4.Bxc6 dxc6 5.d3 Bg4 6.h3 Bxf3 7.Qxf3 Nd7 8.Bd2 g6 9.Bc3 e5 10.a4 Qe7 11.a5 h5 12.Nd2 Bg7 13.h4 Rb8 14.Ke2 O-O 15.Rhg1 b6 16.axb6 axb6 17.Ra7 b5 18.Qh3 f5 19.b3 Qd6 20.Rga1 Rbc8 21.Kf1 Rc7 22.Rxc7 Qxc7 23.Qg3 Kh7 24.Nf3 f4 25.Ng5+ Kh6 26.Qf3 Re8 27.g3 Nf8 28.Bd2 Qd7 29.gxf4 exf4 30.Ra6 Ne6 31.Nxe6 Rxe6 32.Bxf4+ Kh7 33.Be3 Bd4 34.Bxd4 cxd4 35.Ra8 Re8 36.Rxe8 Qxe8 37.Qf6 Qd7 38.e5 Qh3+ 39.Kg1 Qg4+ 40.Kh2 Qd7 41.b4 Qc7 42.Kg2 Qg7 43.Qxg7+ Kxg7 44.Kg3 Kh6 45.f4 Kh7 46.Kf3 Kg8 47.Ke4 Kf7 48.Kxd4 Ke6 49.Ke4 Kd7 50.d4 Kc8 51.e6 Kd8 52.Ke5 c5 53.dxc5 Kc7 54.Kf6 Kc6 55.e7 Kd5 56.Kf7 Kd4 57.e8=Q Kc4 58.c6 Kxb4 59.c7 Ka4 60.c8=Q Ka3 61.Qxb5 Ka2 62.Qca6#

;1.e4 c5 2.Nf3 Nc6 3.d4 cxd4 4.Nxd4 g6 5.Nc3 Bg7 6.Be3 Nf6 7.Nxc6 bxc6 8.e5 Ng8 9.f4 d5 10.Qd4 f6 11.Qa4 e6 12.Qxc6+ Bd7 13.Qa6 Nh6 14.exf6 Qxf6 15.Qb7 Rd8 16.Bc5 Rc8 17.Bb5 Qd8 18.Ne4 Nf5 19.O-O-O Bxb5 20.Qxb5+ Qd7 21.Qxd7+ Kxd7 22.g4 Nh4 23.Ba3 Bf8 24.Nf6+ Kc6 25.Rd3 Ng2 26.Rf1 Bg7 27.Rc3+ Kb5 28.Rg3 Bxf6 29.Rxg2 g5

In [3]:
len_df = df['transcript'].apply(lambda x: len(x))
print(len_df.describe())

# Data setup. All games must have same length. 50% are >= 690 moves. I will discard all games less than 680, and truncate the rest to 680.
filtered_df = df[df['transcript'].apply(lambda x: len(x) >= 680)]
filtered_df['transcript'] = filtered_df['transcript'].apply(lambda x: x[:680])

len_df = filtered_df['transcript'].apply(lambda x: len(x))
print(len_df.describe())

# Now I need to filter out games that are too short. I will discard all games less than 122 moves.
filtered_df = filtered_df[filtered_df['transcript'].apply(lambda x: len(x.split()) > 121)]
print(filtered_df.describe())
print(filtered_df.head())

filtered_df.to_csv('filtered_games.csv', index=False)

move_count_df = filtered_df['transcript'].apply(lambda x: len(x.split()))
print(move_count_df.describe())

count    100000.00000
mean        748.98230
std         352.36122
min          25.00000
25%         539.00000
50%         690.00000
75%         860.00000
max        4366.00000
Name: transcript, dtype: float64
count    51822.0
mean       680.0
std          0.0
min        680.0
25%        680.0
50%        680.0
75%        680.0
max        680.0
Name: transcript, dtype: float64


C:\Users\adamk\AppData\Local\Temp\ipykernel_68240\792275202.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['transcript'] = filtered_df['transcript'].apply(lambda x: x[:680])


                                               transcript
count                                               41206
unique                                              41206
top     ;1.e4 c5 2.Nf3 Nc6 3.d4 cxd4 4.Nxd4 g6 5.Nc3 B...
freq                                                    1
                                           transcript
1   ;1.e4 c5 2.Nf3 Nc6 3.d4 cxd4 4.Nxd4 g6 5.Nc3 B...
2   ;1.e4 e5 2.Nf3 Nc6 3.d4 exd4 4.c3 dxc3 5.Nxc3 ...
9   ;1.e4 c6 2.d4 d6 3.Nf3 Nf6 4.Nc3 Bg4 5.h3 Bh5 ...
12  ;1.e4 c6 2.d4 d6 3.Nf3 Nf6 4.Bd3 e5 5.c3 Nbd7 ...
13  ;1.e4 e5 2.Nf3 Nc6 3.Bb5 Nf6 4.O-O Bd6 5.d4 Nx...
count    41206.000000
mean       123.176965
std          0.918491
min        122.000000
25%        123.000000
50%        123.000000
75%        124.000000
max        128.000000
Name: transcript, dtype: float64


In [4]:
def pretty_print_state_stack(state: np.ndarray) -> None:
    """Given a state stack, print each state in a readable format."""
    piece_symbols = {1: 'W', -1: 'B', 0: '.'}

    # Print the rows in reverse order
    for row in reversed(state):
        print(' '.join(piece_symbols[piece] for piece in row))

def board_to_state(board: chess.Board) -> np.ndarray:
    """Given a chess board object, return a 8x8 np.ndarray.
    The 8x8 array should tell if each square is black, white, or blank.
    White is 1, black is -1, and blank is 0.
    In the 8x8 array, row 0 is A1-H1 (White), row 1 is A2-H2, etc."""
    state = np.zeros((8, 8), dtype=int)
    for i in range(64):
        piece = board.piece_at(i)
        if piece:
            # Assign 1 for white pieces and -1 for black pieces
            state[i // 8, i % 8] = 1 if piece.color == chess.WHITE else -1

    return state

def create_state_stack(moves_string: str) -> np.ndarray:
    """Given a string of PGN format moves, create an 8x8 np.ndarray for every character in the string.
    """

    # pandarallel parallel_apply on windows requires imports to be inside the function being applied
    import chess
    import numpy as np

    # This function is duplicated due to that requirement
    def board_to_state_parallel(board: chess.Board) -> np.ndarray:
        """Given a chess board object, return a 8x8 np.ndarray.
        The 8x8 array should tell if each square is black, white, or blank.
        White is 1, black is -1, and blank is 0.
        In the 8x8 array, row 0 is A1-H1 (White), row 1 is A2-H2, etc."""
        state = np.zeros((8, 8), dtype=np.int8)
        for i in range(64):
            piece = board.piece_at(i)
            if piece:
                # Assign 1 for white pieces and -1 for black pieces
                state[i // 8, i % 8] = 1 if piece.color == chess.WHITE else -1

        return state

    board = chess.Board()
    initial_states = []

    count = 0

    # Scan 1: Creates states, with length = number of moves in the game
    initial_states.append(board_to_state_parallel(board))
    # Apply each move to the board
    for move in moves_string.split():
        # because all games are truncated to len 680, often the last move is partial and invalid
        try:
            count += 1
            # Skip move numbers
            if '.' in move:
                board.push_san(move.split(".")[1])
            else:
                board.push_san(move)

            initial_states.append(board_to_state_parallel(board))
        except:
            break
    # print(board)

        # if count % 100 == 0:
        #     pretty_print_state_stack(state)
        #     print("_" * 50)
        #     print(board)

    # return np.array(initial_states)

    # Second Scan: Expand states to match the length of moves_string
    # For ;1.e4 e5 2.Nf3, ";1.e4" = idx 0, " e5" = idx 1, " 2.Nf3" = idx 2
    expanded_states = []
    move_index = 0
    for char in moves_string:
        if char == ' ':
            move_index += 1
        expanded_states.append(initial_states[min(move_index, len(initial_states) - 1)])

    # expanded_states.append(initial_states[-1]) # The last element in expanded_states is the final position of the board.
    # Currently not using this as len(expanded_states) would be 1 greater than len(moves_string) and that would be confusing.
    return np.array(expanded_states)

df = pd.read_csv('filtered_games.csv')
test_game = df.head()['transcript'][0]
print(test_game[:10])
print(test_game[-10:])
print(len(test_game.split()))
print(len(test_game))
state_stack = create_state_stack(test_game)
print(pretty_print_state_stack(state_stack[-1]))
print(state_stack.shape)

;1.e4 c5 2
.h7 Kd5 62
123
680
. . . . . . . .
. . . . . W W W
. . . . B . B .
. . . B . . . .
. . . . . . . .
. . . . . . . .
. . . W . . . .
. . . . . . . .
None
(680, 8, 8)


In [5]:
def find_dots_indices(moves_string: str) -> list[int]:
    """Returns a NumPy array of indices of every '.' in the string.
    This will hopefully provide a reasonable starting point for training a linear probe."""
    indices = [index for index, char in enumerate(moves_string) if char == '.']
    return indices
    return np.array(indices, dtype=int)

print(find_dots_indices(test_game))


[2, 10, 20, 30, 40, 50, 60, 72, 81, 90, 100, 110, 123, 134, 147, 158, 169, 180, 191, 205, 218, 232, 242, 253, 265, 276, 287, 299, 311, 322, 334, 345, 357, 370, 382, 393, 406, 417, 428, 437, 448, 459, 471, 482, 492, 502, 513, 526, 537, 550, 561, 572, 583, 595, 607, 619, 630, 639, 648, 658, 670]


In [6]:
# Step 2: get meta.pkl, use to create a list of tokenized games with ints
with open('meta.pkl', 'rb') as f:
    meta = pickle.load(f)

print(meta)

stoi, itos = meta["stoi"], meta["itos"]
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode(';1.e4 '))
print(decode(encode(';1.e4 ')))

{'vocab_size': 32, 'itos': {0: ' ', 1: '#', 2: '+', 3: '-', 4: '.', 5: '0', 6: '1', 7: '2', 8: '3', 9: '4', 10: '5', 11: '6', 12: '7', 13: '8', 14: '9', 15: ';', 16: '=', 17: 'B', 18: 'K', 19: 'N', 20: 'O', 21: 'Q', 22: 'R', 23: 'a', 24: 'b', 25: 'c', 26: 'd', 27: 'e', 28: 'f', 29: 'g', 30: 'h', 31: 'x'}, 'stoi': {' ': 0, '#': 1, '+': 2, '-': 3, '.': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, ';': 15, '=': 16, 'B': 17, 'K': 18, 'N': 19, 'O': 20, 'Q': 21, 'R': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'x': 31}}
[15, 6, 4, 27, 9, 0]
;1.e4 


In [16]:
# There are 4 .npy files I will create: One with move strings, one with encoded move token numbers, one with board states, and one with dot indices.
df = pd.read_csv('filtered_games.csv')
row_length = len(df['transcript'].iloc[0])
num_games = len(df)

assert all(df['transcript'].apply(lambda x: len(x) == row_length)), "Not all transcripts are of length {}".format(row_length)

df['transcript'].to_csv('board_seqs_string.csv', index=False, header=False)

# board_seqs_string = np.array(df['transcript'].apply(list).tolist())
print(len(df), len(df['transcript'].iloc[0]))

assert((len(df), len(df['transcript'].iloc[0])) == (num_games, row_length))

41206 680


In [8]:
df = pd.read_csv('filtered_games.csv')
encoded_df = df['transcript'].apply(encode)
print(encoded_df.head())
board_seqs_int = np.array(encoded_df.apply(list).tolist())
print(board_seqs_int.shape)
assert(board_seqs_int.shape == (num_games, row_length))

np.save('board_seqs_int.npy', board_seqs_int)

0    [15, 6, 4, 27, 9, 0, 25, 10, 0, 7, 4, 19, 28, ...
1    [15, 6, 4, 27, 9, 0, 27, 10, 0, 7, 4, 19, 28, ...
2    [15, 6, 4, 27, 9, 0, 25, 11, 0, 7, 4, 26, 9, 0...
3    [15, 6, 4, 27, 9, 0, 25, 11, 0, 7, 4, 26, 9, 0...
4    [15, 6, 4, 27, 9, 0, 27, 10, 0, 7, 4, 19, 28, ...
Name: transcript, dtype: object
(41206, 680)


In [9]:
df = pd.read_csv('filtered_games.csv')
dots_indices_series = df['transcript'].apply(find_dots_indices)

shortest_length = dots_indices_series.apply(len).min()
print("Shortest length:", shortest_length)

dots_indices_series = dots_indices_series.apply(lambda x: x[:shortest_length])
assert all(len(lst) == shortest_length for lst in dots_indices_series), "Not all lists have the same length"

print(dots_indices_series.head())
dots_indices = np.array(dots_indices_series.apply(list).tolist())
print(dots_indices.shape)
assert(dots_indices.shape == (num_games, shortest_length))
np.save('dots_indices.npy', dots_indices)

Shortest length: 61
0    [2, 10, 20, 30, 40, 50, 60, 72, 81, 90, 100, 1...
1    [2, 10, 20, 30, 40, 51, 60, 71, 82, 93, 107, 1...
2    [2, 10, 18, 28, 38, 47, 56, 65, 74, 84, 94, 10...
3    [2, 10, 18, 28, 37, 47, 56, 67, 78, 87, 98, 10...
4    [2, 10, 20, 30, 40, 50, 62, 72, 82, 94, 104, 1...
Name: transcript, dtype: object
(41206, 61)


In [10]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

df = pd.read_csv('filtered_games.csv')
state_stacks_series = df['transcript'].parallel_apply(create_state_stack)
state_stacks = np.stack(state_stacks_series.values)

print(state_stacks.shape)
assert(state_stacks.shape == (num_games, row_length, 8, 8))

np.save('state_stacks.npy', state_stacks)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


(41206, 680, 8, 8)


In [11]:

board_seqs_int = np.load("board_seqs_int_small.npy")
print(board_seqs_int.shape)
print(type(board_seqs_int))

(100000, 60)
<class 'numpy.ndarray'>


In [15]:

import numpy as np

def save_shortened_copy(file_path: str, new_length: int = 5000) -> None:
    """
    Save a shortened copy of a .npy file with a new length.

    Parameters:
    file_path (str): The path to the original .npy file.
    new_length (int): The length of the shortened copy (default is 5000).
    """
    # Load the original data
    data = np.load(file_path)

    # Ensure the new length does not exceed the original length
    new_length = min(new_length, len(data))

    # Shorten the data
    shortened_data = data[:new_length]

    # Create the new filename
    base_name = file_path.split('.')[0]
    new_file_name = f"{base_name}_5k.npy"

    # Save the shortened data
    np.save(new_file_name, shortened_data)

filenames = ['board_seqs_int.npy', 'board_seqs_string.npy', 'dots_indices.npy', 'state_stacks.npy']
# Process each file
for filename in filenames:
    save_shortened_copy(filename)

